In [1]:
import os
import numpy as np
from torch.utils.data import Dataset
import torch
from scipy.signal import butter, filtfilt
from torch.utils.data import random_split

def bandpass_filter(signal):
    fs=100000
    lowcut=50
    highcut=10000
    order=4
    nyquist = 0.5 * fs
    b, a = butter(order, [lowcut / nyquist, highcut / nyquist], btype='band')
    return filtfilt(b, a, signal)
class VibrationSignalDataset(Dataset):
    def __init__(self, root_dir, fs=100000, segment_len=2048):
        self.X, self.y = [], []
        self.fs = fs
        self.segment_len = segment_len

        for label_str in sorted(os.listdir(root_dir)):
            label_path = os.path.join(root_dir, label_str)
            if not os.path.isdir(label_path): continue
            label = int(label_str)  # Assumes folder name = class label
            for fname in os.listdir(label_path):
                if not fname.endswith(".txt"): continue
                fpath = os.path.join(label_path, fname)
                signal =bandpass_filter(np.loadtxt(fpath))
                # Filter dan segmentasi
                segments = [
                    signal[i:i+segment_len]
                    for i in range(0, len(signal[100:]) - segment_len + 1, segment_len)
                ]
                self.X.extend(segments)
                self.y.extend([label]*len(segments))

        self.X = np.array(self.X)
        self.y = np.array(self.y)

    def __len__(self): return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        x = (x - x.mean()) / (x.std() + 1e-8)  # Z-score normalization
        return torch.tensor(x, dtype=torch.float32).unsqueeze(0), torch.tensor(self.y[idx])


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


class CNN_GRU(nn.Module):
    def __init__(self, time_steps, num_classes=3):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),  # (B, 16, L/2)

            nn.Conv1d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),  # (B, 32, L/4)

            nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),  # (B, 64, L/8)

            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),  # Tambahan
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),   # (B, 128, L/16)

            nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1),  # Tambahan
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),   # (B, 128, L/16)

            nn.Conv1d(256, 512, kernel_size=3, stride=1, padding=1),  # Tambahan
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)   # (B, 128, L/16)
        )

        # Hitung ulang time steps setelah pooling (4x maxpool => /16)
        out_time = time_steps // 16

        self.gru = nn.GRU(input_size=512, hidden_size=1024, batch_first=True)

        self.classifier = nn.Sequential(
            nn.Linear(1024, 32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        # x: [batch, 1, time]
        x = self.conv(x)               # => [batch, 128, T/16]
        x = x.permute(0, 2, 1)         # => [batch, seq_len, features]
        out, _ = self.gru(x)           # => [batch, seq_len, 256]
        out = out[:, -1, :]            # Ambil output GRU terakhir
        return self.classifier(out)

def train_model(model, train_loader, val_loader, device, epochs=100):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()

    train_losses = []
    val_losses = []
    val_accuracies = []

    for epoch in range(epochs):
        # --- Train ---
        model.train()
        running_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_losses.append(running_loss / len(train_loader))

        # --- Validation ---
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                loss = criterion(preds, yb)
                val_loss += loss.item()
                _, predicted = torch.max(preds, 1)
                total += yb.size(0)
                correct += (predicted == yb).sum().item()
        val_losses.append(val_loss / len(val_loader))
        val_accuracies.append(correct / total)
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_losses[-1]:.4f} | Val Acc: {val_accuracies[-1]*100:.2f}%")

    return model, train_losses, val_accuracies


In [7]:
from torch.utils.data import DataLoader

# Inisialisasi
dataset = VibrationSignalDataset("/kaggle/input/datasetfix/dataset", fs=100000, segment_len=2048)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Cek satu batch
for xb, yb in train_loader:
    print("Batch X:", xb.shape)  # [batch, 1, 2048]
    print("Batch y:", yb)
    break


Batch X: torch.Size([32, 1, 2048])
Batch y: tensor([0, 1, 1, 0, 0, 1, 2, 2, 1, 2, 2, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 2, 0,
        0, 1, 1, 2, 0, 0, 2, 1])


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(dataset.X, dataset.y, test_size=0.2, stratify=dataset.y)

In [ ]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

# Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_GRU(num_classes=3,time_steps=0.01)  # removed time_steps

# Train
model, train_losses, val_losses, val_accuracies = train_model(model, train_loader, val_loader, device)
torch.save(model.state_dict(), "/kaggle/working/model.pth")
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train Loss")
plt.title("Loss per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

# Plot Akurasi Validasi
plt.subplot(1, 2, 2)
plt.plot(val_accuracies, label="Val Accuracy", color='green')
plt.title("Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.tight_layout()
plt.show()

Epoch 1/100 | Train Loss: 0.8675 | Val Loss: 1.0084 | Val Acc: 60.93%
Epoch 2/100 | Train Loss: 0.7889 | Val Loss: 0.8298 | Val Acc: 60.28%
Epoch 3/100 | Train Loss: 0.7211 | Val Loss: 0.6705 | Val Acc: 71.47%
Epoch 4/100 | Train Loss: 0.7120 | Val Loss: 0.6780 | Val Acc: 73.26%
Epoch 5/100 | Train Loss: 0.6716 | Val Loss: 0.6891 | Val Acc: 69.79%
Epoch 6/100 | Train Loss: 0.6310 | Val Loss: 0.6372 | Val Acc: 69.54%
Epoch 7/100 | Train Loss: 0.5883 | Val Loss: 0.5805 | Val Acc: 74.81%
Epoch 8/100 | Train Loss: 0.5864 | Val Loss: 0.6682 | Val Acc: 73.01%
Epoch 9/100 | Train Loss: 0.5564 | Val Loss: 0.6008 | Val Acc: 76.35%
Epoch 10/100 | Train Loss: 0.5534 | Val Loss: 0.5654 | Val Acc: 76.74%
Epoch 11/100 | Train Loss: 0.5313 | Val Loss: 0.5768 | Val Acc: 77.12%
Epoch 12/100 | Train Loss: 0.5049 | Val Loss: 0.5995 | Val Acc: 77.51%
Epoch 13/100 | Train Loss: 0.4673 | Val Loss: 0.8281 | Val Acc: 73.78%
Epoch 14/100 | Train Loss: 0.4112 | Val Loss: 0.7588 | Val Acc: 70.95%
Epoch 15/100 | 

In [ ]:
torch.save(model.state_dict(), "/kaggle/working/model_euler.pth")
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train Loss")
#plt.plot(val_losses, label="Val Loss")
plt.title("Loss per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

# Plot Akurasi Validasi
plt.subplot(1, 2, 2)
plt.plot(val_accuracies, label="Val Accuracy", color='green')
plt.title("Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.tight_layout()
plt.show()
